# Rekonstruktionsmodelle

## Motivation
Ziel der Arbeit ist eine Evaluation der domänenspezifischen Einsatzfähigkeit des RAVE-Modells zur Aufbereitung und Rekonstruktion von Audiodateien. Die denkbaren Anwendungen sind vielseitig. Die Reduktion auf und die anschließende Rekonstruktion aus einem latenten Raum prädestiniert Autoencoder-Architekturen dazu, in Kompressionsverfahren eingesetzt zu werden, um die Kosten für Datenspeicherung und -Übertragung zu reduzieren. Darüber hinaus sind Szenarien denen Fehlerbehaftete Signale mithilfe von Autoencodern verbessert werden können. Variational Autoencoder können sich in solchen Situationen als besonders nützlich erweisen, da sie sich aufgrund der zusätzlichen Anforderungen an den latenten Raum grundsätzlich besser für Manipulationen der latenten Vektoren eignen. 

Die Einsatzfähigkeit des RAVE-VAEs soll im Folgenden 

## Arten der Signalverfremdung
1. Keine Signalverfremdung
2. Virtuelle Bit-Reduktion auf eine Tiefe von 12 Bits
3. kurze Aussetzer
4. lange Aussetzer

In [ ]:
# Vergleiche Latent Space von unverzerrten Testdaten mit Latent Space von verzerrten 
# Testdaten, um zu verstehen, ob ein Zusammenhang besteht, der für eine Latent Space 
# basierte Verbesserung genutzt werden kann.

import torch
import numpy as np
import pandas as pd
import soundfile as sf
import pathlib
import random

# Load model
MODEL_PATH = pathlib.Path("../../runs/rave_scapes_3/rave_scapes_3.ts")
torch.set_grad_enabled(False)
model = torch.jit.load(MODEL_PATH).eval().double()

originals = pathlib.Path("../../data/soundscapes_multiloc_corr/pollutions/001/test_audio")
bit_reduced = pathlib.Path("../../data/soundscapes_multiloc_corr/pollutions/002/test_audio")
down_sampled = pathlib.Path("../../data/soundscapes_multiloc_corr/pollutions/003/test_audio")

# Sample 10 random data points for rough estimation
random.seed(0)
samples = list(originals.glob("*.wav"))
random.shuffle(samples)
samples = samples[:10]

# Get the latent spaces from originals
ls_orig = []
for f in samples:
    x, sr = sf.read(f)
    z = model.encode(torch.from_numpy(x).reshape(1, 1, -1).double())
    ls_orig.append(z)

# Get latent spaces from bit reduced
ls_bitr = []
for f in samples:
    x, sr = sf.read(bit_reduced / f.name)
    z = model.encode(torch.from_numpy(x).reshape(1, 1, -1).double())
    ls_bitr.append(z)
    
# Get latent spaces from down sampled
ls_down = []
for f in samples:
    x, sr = sf.read(down_sampled / f.name)
    z = model.encode(torch.from_numpy(x).reshape(1, 1, -1).double())
    ls_down.append(z)

In [ ]:
import plotly.express as px

create_latent_space_matrix = lambda x: torch.concat([elem[0].t() for elem in x])
px.bar(create_latent_space_matrix(ls_bitr))
